In [6]:
!pip install -q transformers torch gradio accelerate datasets

In [7]:
import torch
import warnings
from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer,
    AutoTokenizer, AutoModelForSequenceClassification,
    pipeline
)
import gradio as gr
import numpy as np
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

print("Setting up models...")

Setting up models...


In [10]:
class NewsAI:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Load generation model (GPT-2 Medium for better quality)
        print("Loading text generation model...")
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
        self.gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2-medium").to(self.device)

        # Set padding token
        if self.gpt2_tokenizer.pad_token is None:
            self.gpt2_tokenizer.pad_token = self.gpt2_tokenizer.eos_token

        # Load detection model (using a pre-trained fake news detector)
        print("🔍 Loading fake news detection model...")
        try:
            # Try to use a specialized fake news detection model
            self.detector = pipeline(
                "text-classification",
                model="hamzab/roberta-fake-news-classification",
                device=0 if torch.cuda.is_available() else -1
            )
        except:
            # Fallback to BERT-based classification
            print("Specialized model not available, using BERT fallback...")
            self.bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
            self.bert_model = AutoModelForSequenceClassification.from_pretrained(
                "bert-base-uncased", num_labels=2
            ).to(self.device)
            self.detector = None

        print("Models loaded successfully!")

    def generate_fake_news(self, prompt, max_length=300, temperature=0.8):
        """Generate fake news article from prompt"""
        try:
            if not prompt or len(prompt.strip()) < 5:
                return "Please provide a meaningful prompt (at least 5 characters)"

            # Add context to make it more news-like
            enhanced_prompt = f"Breaking News: {prompt.strip()}"

            inputs = self.gpt2_tokenizer.encode(
                enhanced_prompt,
                return_tensors="pt",
                max_length=100,
                truncation=True
            ).to(self.device)

            with torch.no_grad():
                outputs = self.gpt2_model.generate(
                    inputs,
                    max_length=max_length,
                    num_return_sequences=1,
                    no_repeat_ngram_size=3,
                    do_sample=True,
                    temperature=temperature,
                    top_k=40,
                    top_p=0.9,
                    pad_token_id=self.gpt2_tokenizer.eos_token_id,
                    early_stopping=True
                )

            generated_text = self.gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Clean up the text
            generated_text = generated_text.replace(enhanced_prompt, "").strip()
            if not generated_text:
                generated_text = self.gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

            return f"📰 Generated Article:\n\n{enhanced_prompt} {generated_text}"

        except Exception as e:
            return f" Error generating text: {str(e)}"

    def detect_fake_news(self, text):
        """Detect if news is fake or real"""
        try:
            if not text or len(text.strip()) < 10:
                return " Please provide meaningful text (at least 10 characters)"

            if self.detector:
                # Use specialized fake news detector
                result = self.detector(text[:512])  # Limit text length
                label = result[0]['label'].upper()
                confidence = result[0]['score']

                if 'FAKE' in label or 'FALSE' in label:
                    emoji = "🚨"
                    status = "FAKE NEWS"
                else:
                    emoji = "✅"
                    status = "LIKELY REAL"

                return f"{status}\nConfidence: {confidence:.1%}\n\n"

            else:
                # Fallback BERT classification
                inputs = self.bert_tokenizer(
                    text,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512
                ).to(self.device)

                with torch.no_grad():
                    outputs = self.bert_model(**inputs)

                logits = outputs.logits
                probabilities = torch.softmax(logits, dim=1)
                predicted_class = torch.argmax(logits, dim=1).item()
                confidence = probabilities[0][predicted_class].item()

                if predicted_class == 0:
                    return f"🚨 FAKE NEWS\nConfidence: {confidence:.1%}\n\n"
                else:
                    return f"✅ LIKELY REAL\nConfidence: {confidence:.1%}\n\n"

        except Exception as e:
            return f"❌ Error detecting news: {str(e)}"

In [11]:
news_ai = NewsAI()

Using device: cpu
Loading text generation model...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


Models loaded successfully!


In [12]:
def create_interface():
    custom_css = """
    .gradio-container {
        max-width: 1200px !important;
        margin: auto;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 20px;
        border-radius: 15px;
    }
    .tab-nav {
        background: rgba(255,255,255,0.1) !important;
        border-radius: 10px !important;
    }
    .selected {
        background: rgba(255,255,255,0.2) !important;
    }
    .gr-button {
        background: linear-gradient(45deg, #ff6b6b, #ee5a24) !important;
        border: none !important;
        color: white !important;
        font-weight: bold !important;
        border-radius: 25px !important;
        padding: 10px 20px !important;
        transition: transform 0.2s !important;
    }
    .gr-button:hover {
        transform: scale(1.05) !important;
    }
    .gr-textbox {
        border-radius: 10px !important;
        border: 2px solid rgba(255,255,255,0.3) !important;
    }
    .gr-panel {
        background: rgba(255,255,255,0.95) !important;
        border-radius: 15px !important;
        padding: 20px !important;
        margin: 10px 0 !important;
    }
    """

    with gr.Blocks(css=custom_css, title="AI News Generator & Detector") as interface:

        gr.Markdown("""
        # 🤖 AI Fake News Generator & Detector
        ### Educational Tool for Understanding AI-Generated Content

        **⚠️ Disclaimer:** This tool is for educational purposes only. Generated content is fictional and should not be shared as real news.
        """)

        with gr.Tab("🛠️ Generate Fake News", elem_classes="gr-panel"):
            gr.Markdown("### Create AI-generated news content from prompts")

            with gr.Row():
                with gr.Column():
                    gen_input = gr.Textbox(
                        label="📝 Enter News Prompt",
                        placeholder="e.g., Scientists discover new planet, Celebrity spotted doing unusual activity, etc.",
                        lines=3,
                        max_lines=5
                    )

                    with gr.Row():
                        gen_length = gr.Slider(
                            minimum=150,
                            maximum=500,
                            value=300,
                            step=50,
                            label="Article Length"
                        )
                        gen_temp = gr.Slider(
                            minimum=0.5,
                            maximum=1.2,
                            value=0.8,
                            step=0.1,
                            label="Creativity (Temperature)"
                        )

                    gen_button = gr.Button("🚀 Generate Article", variant="primary")

                with gr.Column():
                    gen_output = gr.Textbox(
                        label="Generated Content",
                        lines=15,
                        max_lines=20,
                        placeholder="Generated article will appear here..."
                    )

        with gr.Tab("🔍 Detect Fake News", elem_classes="gr-panel"):
            gr.Markdown("### Analyze text to detect if it might be fake or real")

            with gr.Row():
                with gr.Column():
                    detect_input = gr.Textbox(
                        label="📰 Paste News Article or Text",
                        placeholder="Enter the text you want to analyze...",
                        lines=8,
                        max_lines=10
                    )
                    detect_button = gr.Button("🔍 Analyze Text", variant="primary")

                with gr.Column():
                    detect_output = gr.Textbox(
                        label="Analysis Result",
                        lines=8,
                        placeholder="Analysis results will appear here..."
                    )

        with gr.Tab("ℹ️ Information", elem_classes="gr-panel"):
            gr.Markdown("""
            ## About This Tool

            ### 🎯 Purpose
            This educational tool demonstrates how AI can both generate and detect potentially fake news content.

            ### 🔧 Technology
            - **Generation**: GPT-2 Medium model for text generation
            - **Detection**: RoBERTa model fine-tuned on fake news datasets

            ### ⚠️ Important Notes
            - All generated content is **fictional** and for educational purposes only
            - Detection accuracy is not 100% - always verify information from reliable sources
            - This tool should not be used to create or spread misinformation

            ### 🎓 Educational Value
            - Understand how AI generates human-like text
            - Learn about the challenges of detecting AI-generated content
            - Develop critical thinking skills about information sources

            ### 💡 Tips for Use
            1. **Generation**: Use specific, realistic prompts for better results
            2. **Detection**: Longer texts generally provide more accurate results
            3. **Critical Thinking**: Always cross-reference important information with trusted sources
            """)

        # Event handlers
        gen_button.click(
            fn=lambda prompt, length, temp: news_ai.generate_fake_news(prompt, int(length), temp),
            inputs=[gen_input, gen_length, gen_temp],
            outputs=gen_output
        )

        detect_button.click(
            fn=news_ai.detect_fake_news,
            inputs=detect_input,
            outputs=detect_output
        )

    return interface


In [13]:
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        share=True,
        inbrowser=True,
        show_error=True,
        debug=False
    )
    print("🎉 Interface launched! Click the public URL above to access the app.")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5308a460f655903d06.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎉 Interface launched! Click the public URL above to access the app.
